In [ ]:
!pip install flask-ngrok

In [ ]:
pip install gizeh

In [ ]:
pip install moviepy

In [ ]:
from google.colab import drive, files           #import drive
drive.mount('/content/MyDrive',force_remount=False)
%cd '/content/MyDrive/My Drive/'

Mounted at /content/MyDrive
/content/MyDrive/My Drive


In [ ]:
#import necessary libraries and files
import os
import cv2
import glob
import gizeh
import requests
import gizeh as gz
from PIL import Image
import moviepy.editor as mp               
from moviepy.editor import *
from flask_ngrok import run_with_ngrok
from flask import Flask,request,json,jsonify
from Tikact_video.crop1 import cropped_img
from Tikact_video.end_text import texts_end
from Tikact_video.logo1 import logo
from Tikact_video.user_text1 import logo_text                 
from Tikact_video.dp_logo4 import end_dp_logo
from Tikact_video.image_download1 import image
from Tikact_video.video_download2 import video_download

In [ ]:
gif_dir="logo.gif"                         #logo_directory
output_dir="videos/" 
input_dir="input/"    
end_image='watermark-last.gif' 

In [ ]:
app=Flask(__name__)               
run_with_ngrok(app)
                       #use_ngork to run flask in colab        
@app.route('/')
def index():
  return "Tikact"

@app.route('/api/uploads', methods = ['POST'])   #rout
def uploads():
  if request.method =='POST':
    content=request.get_json()              #get_json
    username=content['username']            #get_username
    user_pro_name=content['user_pro_name']  #get_user_profilename
    video_url=content['video_url']          #get_video_url and download it
    dp_image_url=content['dp']              #get_display picture

    #for video downloading
    video_dir=video_download(video_url,input_dir+'simple.mp4' )
    #for image_downloading
    dp_image=image(dp_image_url,input_dir+'image.png')

    #storing other variables
    video = mp.VideoFileClip(video_dir)     #assaign video into a variable in moviepy
    duration=video.duration                 #totsl duration of video
    half_duration=duration/2.01             #half duration
    last_duration=3   #3 sec                #duartion of last screen
    W=video.w                               #Width of video
    H=video.h                               #Height of video
    pixel_size=video.size                   #resolution
    print(pixel_size)

    #condition 
    if W > H:
      logo_size=W*.06   #.06
      logo_text_size=W*.022
      username_size=W*.02
      user_pro_name_size=W*.03
      dp_image_size=W*.093                  #condition for landscape videos
      logo_end_size=W*.2  #.18
    else:
      logo_size=H*.06
      logo_text_size=H*.022
      username_size=H*.02
      user_pro_name_size=H*.03             #condition for portrait videos
      dp_image_size=H*.093
      logo_end_size=H*.2 

    #Logo applying
    return_logo=logo(W,H,gif_dir=gif_dir,logo_size=logo_size)
    logo_left=return_logo[0]
    logo_right=return_logo[1]               #access the return values

    #username with logo applying
    logo_texts=logo_text(W,H,username,video_dir,logo_text_size,half_duration,logo_size) 
                        #taking current file
    original_clip=logo_texts[0]
    graphics_clip_left=logo_texts[1]
    graphics_clip_right=logo_texts[2]      #access the return values

    # applying for crop of downloaded image into circle
    crop_image=cropped_img(input_dir=dp_image,output_dir=output_dir+'dp'+'.png')

    #end_screen texts that username and user pro name are applying
    text_end=texts_end(W,H,username,username_size,user_pro_name,video_dir,dp_image_size,user_pro_name_size,last_duration)
    original_clip=text_end[0]
    graphics_clip_end=text_end[1]       #access the return value

    #dp_image and end_logo applying
    dp_and_logo=end_dp_logo(W,H,crop_image,end_image,dp_image_size,logo_end_size)
    dp=dp_and_logo[0]
    logo_end =dp_and_logo[1]

    #final composite of video applying
    final_clip=	CompositeVideoClip(
            [original_clip,
            graphics_clip_left.set_start(0).set_duration(half_duration),
            graphics_clip_right.set_start(half_duration).set_duration(half_duration),
            logo_left.set_start(0).set_duration(half_duration),
            logo_right.set_start(half_duration).set_duration(half_duration),
            graphics_clip_end.set_start(duration).set_duration(last_duration),
            dp.set_start(duration).set_duration(last_duration),
            logo_end.set_start(duration).set_duration(last_duration)],
            size=(W, H))
  
    #invoking for final video
    final_clip.write_videofile(output_dir+"new_one_test.mp4",
                               fps=24,
                               threads=4,
                               progress_bar=False,
                               codec = 'libx264',
                               temp_audiofile='temp-audio.m4a',
                               remove_temp=True,audio_codec="aac",
                               preset='medium')
    return "video Downloaded"
        
    
      


if __name__ =="__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://12a89926d462.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Oct/2020 12:24:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Oct/2020 12:24:43] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:

# https://vod-progressive.akamaized.net/exp=1602070958~acl=%2Fvimeo-prod-skyfire-std-us%2F01%2F3099%2F15%2F390497693%2F1651265064.mp4~hmac=31e16de5d5cc644dd910d91ba71d44638c26445d85a0d431c71029da9a7f874f/vimeo-prod-skyfire-std-us/01/3099/15/390497693/1651265064.mp4?filename=Love+-+32021.mp4

#https://www.pexels.com/video/1433307/download/?search_query=nature&tracking_id=fbqyn6wovki&w=1280&h=720

'/content/MyDrive/My Drive/input/video3.mp4'